# Excelで入力した教師データの拡張

編集日：2023/12/03 <br>
編集者：[nogikun](https://github.com/nogikun) <br>

#### 期待するファイル
以下のようなテーブルを含むExcelファイルを用意してください。<br>
シート数は１つで、シート名は任意です。

※ `system` はA1セル、`user` はB1セル、`assistant` はC1セルに入力してください。

|system|user|assistant|
|:-:|:-:|:-:|
|あなたはスタックチャンです。おはなしをしよう。|こんにちは|こんにちは僕の名前はスタックチャンです|
|あなたはスタックチャンです。おはなしをしよう。|{任意の質問}|{任意の質問に対応する解答}|
|︙|︙|︙|


#### 実行